# Propositional Logic Experiment

In this notebook, we will test your skills on using the LNN in a propositional setting. You will be required to use your knowledge about per-node reasoning and bounds to complete the code and answer the query. There is a very similar problem at [url], and if you complete that, you will be able to apply for the IBM Neurosymbolic Essentials Badge.

### Instructions

1. _Use_ the illustrations given as a guide on how to reason with an LNN graph

2. _Fill_ in code that will evaluate the correct truth values expected from performing logical inference

3. _Query_ the logical disjunction between nodes C, D and E

### Propositional Inference

Recall that a proposition is any declarative sentence that allows you to associate a truth value with the atomic formulae. An LNN graph can hierarchically compose these propositions using logical connectives and reason over their respective truths. Evaluating a particular node requires an upward/downward inference pass.

Suppose that you are given the following LNN graph:


where the following beliefs are outputs from sensor information:
```
root: TRUE
A: 0.9
B: False
E: 0.2 - 0.8
```

Your task is to perform successive inference passes to obtain the truth value of a query node. 
> HINT: use the `node.print()` function as a guide to ensure that the LNN is returning the correct truths. 

The final query is to return the truth of a **disjunction** of the following propositions:

\begin{equation}(C, D, E)\end{equation}

> NB: use the provided function definition to help you extract the correct truth value bounds

<center>
    <figure>
        <img src="./img/experiment/0/graph.png" width="400" />
        <figcaption>Figure 1. Propositional inference</figcaption>
    </figure>
</center>

Fill in the codeblock below to construct the graph and reason about all nodes:

In [1]:
from lnn import Propositions, And, Implies, Iff, Fact, Model, Or

# Knowledge - Set up the propositions A, B, C, D, E.
# Notebook 1 will be useful for this.
# Set up the subpropositions and the full proposition.
A, B, C, D, E = Propositions("A", "B", "C", "D", "E")
IMPLIES=Implies(A, B)
AND=And(C, D)
IFF=Iff(AND, E)
SENTENCE =And(IMPLIES, IFF)

# Data - Add the truth values of each node as described above.
# Print the values of each leaf node and the sentence as a whole.
SENTENCE.add_data(Fact.TRUE)
A.add_data(0.9)
B.add_data(Fact.FALSE)
E.add_data((0.2, 0.8))
SENTENCE.print()
A.print()
B.print()
C.print()
D.print()
E.print()

# Set up a model and add the knowledge we have
model=Model()
model.add_knowledge(SENTENCE)
model.add_knowledge(IFF)
model.add_knowledge(AND)
model.add_knowledge(IMPLIES)
model.add_knowledge(A)
model.add_knowledge(B)
model.add_knowledge(C)
model.add_knowledge(D)
model.add_knowledge(E)

# Reasoning
model.infer()
model.print(params=True)

OPEN And: ((A → B) ∧ (((C ∧ D) → E) ∧ (E → (C ∧ D))))          TRUE (1.0, 1.0)

OPEN Proposition: A                                  APPROX_TRUE (0.9, 0.9)

OPEN Proposition: B                                        FALSE (0.0, 0.0)

OPEN Proposition: C                                      UNKNOWN (0.0, 1.0)

OPEN Proposition: D                                      UNKNOWN (0.0, 1.0)

OPEN Proposition: E                                APPROX_UNKNOWN (0.2, 0.8)


***************************************************************************
                                LNN Model

OPEN And: ((A → B) ∧ (((C ∧ D) → E) ∧ (E → (C ∧ D)))) CONTRADICTION (1.0, 0.1)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Iff: (((C ∧ D) → E) ∧ (E → (C ∧ D)))                UNKNOWN (0.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Implies: (E → (C ∧ D))                        APPROX_UNKNOWN (0.2, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Implies: ((C ∧ D) → E)                        APPROX_UNKNOWN (0.2,

### Query
Finally, find the state of the **disjunction** between nodes `B`, `D` and `E` using the `query_state` method.

<center>
    <figure>
        <img src="./img/experiment/0/query.png" width="200" />
        <figcaption>Figure 5. Perform inference to find the disjunction between nodes B, D and E</figcaption>
    </figure>
</center>

In [2]:
def query_state(node):
    round_off = lambda my_list: [float(f"{_:.1f}") for _ in my_list]
    return f"{node.state().name}: {tuple(round_off(node.get_data().tolist()))}"

Fill in the codeblock below to extract the solution to the query:

In [3]:
query = Or(C,D,E)

query.upward()

query_state(query)

'APPROX_UNKNOWN: (0.2, 1.0)'

## Training
Earlier, you should have seen that the data we gave the model is contradictory. We can train model weights to obtain the output we want. 

By introducing a loss on the contradiction:

\begin{align}
 Loss = \lambda \sum_{\forall i} \text{max} (0, L_i - U_i)
\end{align}

we can train the LNN to adjust the weights such that all sources of contradictions become downweighted.

In [4]:
from lnn import Model, Loss, Direction

# Train the model in the upward direction and print out the model 
# and its parameters. How have the model weights changed?
model.train(direction=Direction.UPWARD, losses=[Loss.CONTRADICTION])
model.print(params=True)


***************************************************************************
                                LNN Model

OPEN And: ((A → B) ∧ (((C ∧ D) → E) ∧ (E → (C ∧ D))))          TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [0.55865 1.     ]
OPEN Iff: (((C ∧ D) → E) ∧ (E → (C ∧ D)))            APPROX_TRUE (0.5871, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Implies: (E → (C ∧ D))                          APPROX_TRUE (0.79355, 1.0)
params  α: 1.0,  β: 0.50538,  w: [1.49462 1.49462]
OPEN Implies: ((C ∧ D) → E)                          APPROX_TRUE (0.79355, 1.0)
params  α: 1.0,  β: 0.50538,  w: [1.49462 1.49462]
OPEN Proposition: E                                APPROX_UNKNOWN (0.2, 0.8)
params  α: 1.0
OPEN And: (C ∧ D)                                        UNKNOWN (0.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [2.00773 2.00773]
OPEN Proposition: D                                      UNKNOWN (0.0, 1.0)
params  α: 1.0
OPEN Proposition: C                                      UNKNOWN (0.0,

/opt/anaconda3/envs/week4_lnn/lib/python3.9/site-packages/torch/optim/adam.py:137: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)


Print out the values of each node in the model. Have they changed? How?

In [5]:
SENTENCE.print()
AND.print()
IFF.print()
IMPLIES.print()
A.print()
B.print()
C.print()
D.print()
E.print()

OPEN And: ((A → B) ∧ (((C ∧ D) → E) ∧ (E → (C ∧ D))))          TRUE (1.0, 1.0)

OPEN And: (C ∧ D)                                        UNKNOWN (0.0, 1.0)

OPEN Iff: (((C ∧ D) → E) ∧ (E → (C ∧ D)))            APPROX_TRUE (0.5871, 1.0)

OPEN Implies: (A → B)                                       TRUE (1.0, 1.0)

OPEN Proposition: A                                  APPROX_TRUE (0.9, 0.9)

OPEN Proposition: B                                        FALSE (0.0, 0.0)

OPEN Proposition: C                                      UNKNOWN (0.0, 1.0)

OPEN Proposition: D                                      UNKNOWN (0.0, 1.0)

OPEN Proposition: E                                APPROX_UNKNOWN (0.2, 0.8)



In [6]:
query = Or(C,D,E)

query.upward()

query_state(query)

'APPROX_UNKNOWN: (0.2, 1.0)'

In [7]:
model.train(direction=Direction.DOWNWARD, losses=[Loss.CONTRADICTION])
model.print(params=True)


***************************************************************************
                                LNN Model

OPEN And: ((A → B) ∧ (((C ∧ D) → E) ∧ (E → (C ∧ D))))          TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [0.55865 1.     ]
OPEN Iff: (((C ∧ D) → E) ∧ (E → (C ∧ D)))                   TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Implies: (E → (C ∧ D))                                 TRUE (1.0, 1.0)
params  α: 1.0,  β: 0.50538,  w: [1.49462 1.49462]
OPEN Implies: ((C ∧ D) → E)                                 TRUE (1.0, 1.0)
params  α: 1.0,  β: 0.50538,  w: [1.49462 1.49462]
OPEN Proposition: E                                APPROX_UNKNOWN (0.2, 0.8)
params  α: 1.0
OPEN And: (C ∧ D)                                        UNKNOWN (0.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [2.00773 2.00773]
OPEN Proposition: D                                      UNKNOWN (0.0, 1.0)
params  α: 1.0
OPEN Proposition: C                                      UNKNOWN (0.0, 1.0)
param

In [8]:
SENTENCE.print()
AND.print()
IFF.print()
IMPLIES.print()
A.print()
B.print()
C.print()
D.print()
E.print()

OPEN And: ((A → B) ∧ (((C ∧ D) → E) ∧ (E → (C ∧ D))))          TRUE (1.0, 1.0)

OPEN And: (C ∧ D)                                        UNKNOWN (0.0, 1.0)

OPEN Iff: (((C ∧ D) → E) ∧ (E → (C ∧ D)))                   TRUE (1.0, 1.0)

OPEN Implies: (A → B)                                       TRUE (1.0, 1.0)

OPEN Proposition: A                                  APPROX_TRUE (0.9, 0.9)

OPEN Proposition: B                                        FALSE (0.0, 0.0)

OPEN Proposition: C                                      UNKNOWN (0.0, 1.0)

OPEN Proposition: D                                      UNKNOWN (0.0, 1.0)

OPEN Proposition: E                                APPROX_UNKNOWN (0.2, 0.8)



In [9]:
model.print(params=True)


***************************************************************************
                                LNN Model

OPEN And: ((A → B) ∧ (((C ∧ D) → E) ∧ (E → (C ∧ D))))          TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [0.55865 1.     ]
OPEN Iff: (((C ∧ D) → E) ∧ (E → (C ∧ D)))                   TRUE (1.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [1. 1.]
OPEN Implies: (E → (C ∧ D))                                 TRUE (1.0, 1.0)
params  α: 1.0,  β: 0.50538,  w: [1.49462 1.49462]
OPEN Implies: ((C ∧ D) → E)                                 TRUE (1.0, 1.0)
params  α: 1.0,  β: 0.50538,  w: [1.49462 1.49462]
OPEN Proposition: E                                APPROX_UNKNOWN (0.2, 0.8)
params  α: 1.0
OPEN And: (C ∧ D)                                        UNKNOWN (0.0, 1.0)
params  α: 1.0,  β: 1.0,  w: [2.00773 2.00773]
OPEN Proposition: D                                      UNKNOWN (0.0, 1.0)
params  α: 1.0
OPEN Proposition: C                                      UNKNOWN (0.0, 1.0)
param

In [10]:
A, B= Propositions("A", "B")
IMPLIES=Implies(A, B)
A.add_data(0.9)
B.add_data(Fact.FALSE)
A.print()
B.print()
IMPLIES.print()
model = Model()
model.add_knowledge(IMPLIES)
model.infer()
IMPLIES.print()

OPEN Proposition: A                                  APPROX_TRUE (0.9, 0.9)

OPEN Proposition: B                                        FALSE (0.0, 0.0)

OPEN Implies: (A → B)                                    UNKNOWN (0.0, 1.0)

OPEN Implies: (A → B)                               APPROX_FALSE (0.1, 0.1)



# Predicate Logic

We will revisit the friends and movies example from Week 3. We use a simplifed setup:

We will have 4 users: $a$, $b$, $c$, $d$.

and 2 films: $j$, $l$.

**Variables** are $x$, with domain $people$, $y$ with domain $people$ and $u$ with domain $films$

We will have **predicates** $F(x,y)$ for *friends* and $L(x, u)$ for *likes*

**Knowledge**

- $\forall xyu (F(x, y) \land L(x, u)) \implies L(y, u)$

**Data**

$\mathcal{F} = \{(a, b), (a, c), (b,c), (c, d)\}$

$\mathcal{L} = \{(a, j), (a, l),  (b, l), (c, l), (d, j)\}$

- $F(x, y)$ for $(x, y) \in \mathcal{F}$
- $\neg F(x, y)$ for $(x, y) \not\in \mathcal{F}$
- $L(x, u)$ for $(x, u) \in \mathcal{L}$
- $\neg L(c, j)$

We can see this is not strictly logically satisfiable, since we have $F(a, c)$, $L(a, j)$, but also $\neg L(c, j)$.

We start by initializing an empty model:


In [22]:
# Initialize an empty model
from lnn import Model

model = Model()

Then we go on to set up predicates and variables. 

In [23]:
# Set up predicates for our model
from lnn import Predicates, Variables

Friends = Predicates('Friends', arity=2)
Likes = Predicates('Likes', arity=2)
x, y, u = Variables('x', 'y', 'u')

Encode the knowledge (see above) about the world using neural connectives.

In [24]:
from lnn import Not, And, Implies

Friends_like_Films = Implies(And(Friends(x, y), Likes(x, u)), Likes(y, u))


Add the knowledge to the model with a closed world assumption. Have a look at the Predicate Logic notebook for a hint.

In [25]:
from lnn import World

formulae = [
    Friends_like_Films
]
model.add_knowledge(*formulae, world=World.AXIOM)

We now need to add facts to the model. Which pairs of people are friends? Which people like which films? Have a look at the Predicate Logic notebook for a hint.

In [26]:
from lnn import Fact

# add data to the model
model.add_data({
    Friends: {
        ('a', 'b'):Fact.TRUE,
        ('a', 'c'):Fact.TRUE,
        ('b', 'c'):Fact.TRUE,
        ('c', 'd'):Fact.TRUE,
    }, 
    Likes: {
        ('a', 'j'):Fact.TRUE,
        ('a', 'l'):Fact.TRUE,
        ('b', 'l'):Fact.TRUE,
        ('c', 'l'):Fact.TRUE,
        ('d', 'j'):Fact.TRUE,
        ('c', 'j'):Fact.FALSE,
    }, 
    })
model.print()


***************************************************************************
                                LNN Model

AXIOM Implies: ((Friends(0, 1) ∧ Likes(0, 2)) → Likes(1, 2)) 

OPEN And: (Friends(0, 1) ∧ Likes(0, 2)) 

OPEN Predicate: Likes 
('c', 'l')                                                  TRUE (1.0, 1.0)
('a', 'j')                                                  TRUE (1.0, 1.0)
('b', 'l')                                                  TRUE (1.0, 1.0)
('c', 'j')                                                 FALSE (0.0, 0.0)
('a', 'l')                                                  TRUE (1.0, 1.0)
('d', 'j')                                                  TRUE (1.0, 1.0)

OPEN Predicate: Friends 
('b', 'c')                                                  TRUE (1.0, 1.0)
('a', 'c')                                                  TRUE (1.0, 1.0)
('c', 'd')                                                  TRUE (1.0, 1.0)
('a', 'b')                                

Run inference over the model and print the model out. What do you see?

In [27]:
model.infer()
model.print(params=True)


***************************************************************************
                                LNN Model

AXIOM Implies: ((Friends(0, 1) ∧ Likes(0, 2)) → Likes(1, 2)) 
params  α: 1.0,  β: 1.0,  w: [1. 1.]
('a', 'a', 'j')                                             TRUE (1.0, 1.0)
('d', 'c', 'j')                                             TRUE (1.0, 1.0)
('c', 'b', 'l')                                             TRUE (1.0, 1.0)
('b', 'a', 'j')                                             TRUE (1.0, 1.0)
('b', 'c', 'l')                                             TRUE (1.0, 1.0)
('c', 'c', 'j')                                             TRUE (1.0, 1.0)
('d', 'b', 'l')                                             TRUE (1.0, 1.0)
('c', 'd', 'l')                                             TRUE (1.0, 1.0)
('a', 'b', 'l')                                             TRUE (1.0, 1.0)
('a', 'c', 'j')                                    CONTRADICTION (1.0, 0.0)
('b', 'b', 'l')      

Train the model using the contradiction loss (have a look at the notebooks to remind yourself of this).

In [28]:
from lnn import Direction, Loss
model.train(losses=Loss.CONTRADICTION)

(([3.0,
   2.75,
   2.5,
   2.249999761581421,
   1.999999761581421,
   1.749999761581421,
   1.4999996423721313,
   1.2499995231628418,
   0.9999995231628418,
   0.749999463558197,
   0.49999988079071045,
   0.44999992847442627,
   0.3999999761581421,
   0.34999996423721313,
   0.2999999523162842,
   0.25,
   0.19999998807907104,
   0.1499999761581421,
   0.10000002384185791,
   0.050000011920928955,
   0.0],
  [[3.0],
   [2.75],
   [2.5],
   [2.249999761581421],
   [1.999999761581421],
   [1.749999761581421],
   [1.4999996423721313],
   [1.2499995231628418],
   [0.9999995231628418],
   [0.749999463558197],
   [0.49999988079071045],
   [0.44999992847442627],
   [0.3999999761581421],
   [0.34999996423721313],
   [0.2999999523162842],
   [0.25],
   [0.19999998807907104],
   [0.1499999761581421],
   [0.10000002384185791],
   [0.050000011920928955],
   [0.0]]),
 [0.0,
  17.5,
  16.999998092651367,
  16.5,
  16.0,
  15.5,
  14.999999046325684,
  14.499998092651367,
  14.0,
  13.49999904632

Print out the model after training. What do you notice has changed from before training?

In [29]:
model.print(params=True)


***************************************************************************
                                LNN Model

AXIOM Implies: ((Friends(0, 1) ∧ Likes(0, 2)) → Likes(1, 2)) 
params  α: 1.0,  β: 0.0,  w: [1. 1.]
('a', 'a', 'j')                                             TRUE (1.0, 1.0)
('d', 'c', 'j')                                             TRUE (1.0, 1.0)
('c', 'b', 'l')                                             TRUE (1.0, 1.0)
('b', 'a', 'j')                                             TRUE (1.0, 1.0)
('b', 'c', 'l')                                             TRUE (1.0, 1.0)
('c', 'c', 'j')                                             TRUE (1.0, 1.0)
('d', 'b', 'l')                                             TRUE (1.0, 1.0)
('c', 'd', 'l')                                             TRUE (1.0, 1.0)
('a', 'b', 'l')                                             TRUE (1.0, 1.0)
('a', 'c', 'j')                                             TRUE (1.0, 1.0)
('b', 'b', 'l')      